# TFG FER

Este notebook contiene el código necesario para la creación de un modelo de clasificación de imágenes de rostros humanos en función del estres emocional que presentan. Para ello, se va a utuilzar un dataset de imágenes pero de tipo .npy que contiene las imágenes de los rostros pero con un formato diferente al habitual, ya que las imágenes están en formato de matriz de píxeles. 

Los pasos a seguir para la creación del modelo son los siguientes:

1. Importar las librerías necesarias
2. Cargar el dataset
3. Preprocesamiento de los datos
4. Creación del modelo
5. Entrenamiento del modelo
6. Evaluación del modelo

In [1]:
import os
import tensorflow as tf

# Configurar TensorFlow para que use la CPU
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Ahora, al crear sesiones de TensorFlow, solo se usará la CPU
# Verificar que realmente no estamos utilizando ninguna GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


2024-07-17 13:34:48.177643: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-17 13:34:48.933149: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/fgalan/miniconda3/lib/:/home/fgalan/miniconda3/lib/:/home/fgalan/miniconda3/envs/myenv/lib/
2024-07-17 13:34:48.933258: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/fgalan/miniconda3/lib/:/

Num GPUs Available:  2


2024-07-17 13:34:50.007497: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-17 13:34:50.007814: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-17 13:34:50.016756: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-17 13:34:50.016998: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-17 13:34:50.017217: I tensorflow/compiler/xla/stream_executo

## 1. Importar las librerías necesarias

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
import os
import random

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

# Comprobar version tensorflow
print(tf.__version__)

# Comprobar que estamos usando la GPU
print(tf.config.list_physical_devices())

2.11.0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


## 2. Cargar el dataset

Como se ha mencionado anteriormente, el dataset que se va a utilizar es un archivo .npy que contiene las imágenes de los rostros en formato de matriz de píxeles. Para cargar el dataset, se utiliza la función `np.load()` de la librería NumPy.

Nuestro dataset esta dividido en dos partes, una con las imágenes y otra con las etiquetas. Para la recolección de imágenes se ha utilizado un dataset cuyo escenario experimental consiste en pasos que causan o no causan estrés; lectura de escritura en el idioma nativo, entrevista en el idioma nativo, lectura de escritura en idioma no nativo, entrevista en idioma no nativo.

Por lo tanto las distintas carpetas que forma el dataset son:

- **Native_Language_Script_Reading** (BAJO ESTRES-NEUTRAL)
- **Native_Language_Interview** (BAJO ESTRES)
- **Non_Native_Language_Script_Reading** (ALTO ESTRES-NEUTRAL)
- **Non_Native_Language_Interview** (ALTO ESTRES)

Para procesar archivos .npy de la manera que se ha descrito y asignar las etiquetas correspondientes (como "BAJO ESTRES-NEUTRAL", "BAJO ESTRES", "ALTO ESTRES-NEUTRAL", "ALTO ESTRES") basadas en el nombre de la carpeta en la que se encuentran, se van a seguir los siguientes pasos:

### 2.1 Asignar las etiquetas a las imágenes

Para asignar las etiquetas a las imágenes, se va a utilizar una primero una función que recorra las carpetas del dataset y asigne las etiquetas correspondientes a cada imagen.

In [3]:
def asignar_etiqueta(nombre_carpeta):
    if nombre_carpeta == "Native_Language_Interview":
        return "BAJO ESTRES"
    elif nombre_carpeta == "Native_Language_Script_Reading":
        return "BAJO ESTRES-NEUTRAL"
    elif nombre_carpeta == "Non-native_Language_Interview":
        return "ALTO ESTRES"
    elif nombre_carpeta == "Non-native_Language_Script_Reading":
        return "ALTO ESTRES-NEUTRAL"
    else:
        return None  # Por si el nombre de la carpeta no coincide

### 2.2 Cargar las imágenes y las etiquetas

Para el dataset que se va a utilizar, se van a cargar las imágenes de distintas carpetas, segun la persona que aparece en la imagen, y las etiquetas correspondientes a cada imagen. Dentro de cada carpeta de cada persona, se encuentran las carpetas con las clases de estrés correspondientes, y dentro de cada una de estas carpetas se encuentran las imágenes. Lo que se va a realizar es cargar las imágenes y las etiquetas de cada una de las carpetas de las clases de estrés, y se van a guardar todas las corresponientes a cada persona en un array de numpy. Para asi tener un array de numpy con todas las imágenes y otro con todas las etiquetas.

In [4]:
# Definir la ruta base de tu DATASET
ruta = "DATASET"

# Definir las subcarpetas que corresponden a cada clase
classes = ["Native_Language_Script_Reading", "Native_Language_Interview",
           "Non-native_Language_Script_Reading", "Non-native_Language_Interview"]

# Número deseado de imágenes por clase
num_images_per_class = 500

# Listas para almacenar las imágenes y las etiquetas
imagenes = []
etiquetas = []

# Recorrer cada persona en el dataset
for carpeta_persona in os.listdir(ruta):
    ruta_persona = os.path.join(ruta, carpeta_persona)
    print(ruta_persona)
    if os.path.isdir(ruta_persona):  # Asegurar que es un directorio
        # Recorrer cada clase
        for cls in classes:
            ruta_clase = os.path.join(ruta_persona, cls)
            if os.path.exists(ruta_clase):
                # Lista todas las imágenes (archivos .npy) en la carpeta de la clase
                images = [os.path.join(ruta_clase, img) for img in os.listdir(ruta_clase) if img.endswith('.npy')]            
                # Seleccionar 1000 imágenes aleatorias sin repetición
                if len(images) >= num_images_per_class:
                    selected_images = random.sample(images, num_images_per_class)
                else:
                    selected_images = images  # Si hay menos de 1000, tomar todas
                
                # Añadir las imágenes seleccionadas y sus etiquetas a las listas
                for ruta_imagen in selected_images:
                    imagen = np.load(ruta_imagen, allow_pickle=True)
                    imagenes.append(imagen)
                    etiquetas.append(asignar_etiqueta(cls))
                    
# Contar etiquetas por clase
conteo_etiquetas = {etiqueta: etiquetas.count(etiqueta) for etiqueta in set(etiquetas)}

# Número de etiquetas creadas por clase
for etiqueta, conteo in conteo_etiquetas.items():
    print(f"Etiqueta '{etiqueta}': {conteo} imágenes")



DATASET/05


DATASET/28
DATASET/20
DATASET/29
DATASET/22
DATASET/30
DATASET/10
DATASET/14
DATASET/21
DATASET/02
DATASET/25
DATASET/11
DATASET/15
DATASET/16
DATASET/09
DATASET/33
DATASET/17
DATASET/24
DATASET/32
DATASET/18
DATASET/27
DATASET/03
DATASET/23
DATASET/19
DATASET/13
DATASET/01
DATASET/07
DATASET/08
DATASET/31
DATASET/04
DATASET/12
DATASET/06
Etiqueta 'ALTO ESTRES-NEUTRAL': 16000 imágenes
Etiqueta 'BAJO ESTRES-NEUTRAL': 16000 imágenes
Etiqueta 'ALTO ESTRES': 16000 imágenes
Etiqueta 'BAJO ESTRES': 16000 imágenes


### 2.3 Visualizar las imágenes

Antes de continuar con el preprocesamiento de los datos, se van a visualizar algunas de las imágenes del dataset para comprobar que se han cargado correctamente.

In [5]:
# Comprobar que se han cargado las imágenes
print("Número de imágenes cargadas:", len(imagenes))
print("Número de etiquetas cargadas:", len(etiquetas))


Número de imágenes cargadas: 64000
Número de etiquetas cargadas: 64000


Vamos a visualizar el número de imágenes que hay en cada categoría

In [6]:
# Contar cuántas imágenes hay de cada etiqueta
from collections import Counter

conteo = Counter(etiquetas)
print("Número de imágenes por etiqueta:", conteo)  
    

Número de imágenes por etiqueta: Counter({'BAJO ESTRES-NEUTRAL': 16000, 'BAJO ESTRES': 16000, 'ALTO ESTRES-NEUTRAL': 16000, 'ALTO ESTRES': 16000})


### 2.4 Preprocesamiento de los datos

Una vez que se han cargado las imágenes y las etiquetas, se van a preprocesar los datos para poder utilizarlos en el modelo de clasificación. Para ello, se van a seguir los siguientes pasos:

- Normalizar las imágenes
- Dividir los datos en conjuntos de entrenamiento y prueba
- Codificar las etiquetas

#### 2.4.1 Normalizar las imágenes

In [7]:
# Convertir la lista de imágenes y etiquetas a un array de Numpy para facilitar su manipulación
imagenes = np.array(imagenes, dtype="float32") / 255
print(imagenes.shape)

etiquetas = np.array(etiquetas)
print(etiquetas.shape)

(64000, 256, 7, 7)
(64000,)


#### 2.4.2 Dividir los datos en conjuntos de entrenamiento y prueba

Dividimos el conjunto de entrenamiento, conjunyo de test y conjunto de validación, con una proporción de 3:1:1 respectivamente.

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Codificar las etiquetas
label_encoder = LabelEncoder()
etiquetas_encoded = label_encoder.fit_transform(etiquetas)  # Codificar todas las etiquetas
etiquetas_categorical = to_categorical(etiquetas_encoded)  # Convertir a one-hot

# Dividir los datos en datos de entrenamiento y temporales (combinando validación y prueba)
X_train, X_temp, y_train, y_temp = train_test_split(imagenes, etiquetas_categorical, test_size=0.4, random_state=42)

# Dividir los datos temporales en datos de validación y prueba
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Comprobar el tamaño de los conjuntos de datos
print("Datos de entrenamiento:", X_train.shape, y_train.shape)
print("Datos de validación:", X_val.shape, y_val.shape)
print("Datos de prueba:", X_test.shape, y_test.shape)

Datos de entrenamiento: (38400, 256, 7, 7) (38400, 4)
Datos de validación: (12800, 256, 7, 7) (12800, 4)
Datos de prueba: (12800, 256, 7, 7) (12800, 4)


Esta manera de dividir los datos nos va a permitir entrenar el modelo con el conjunto de entrenamiento, ajustar los hiperparámetros con el conjunto de validación y evaluar el modelo con el conjunto de prueba.

Tambien es crucial para evitar el overfitting, ya que si entrenamos y evaluamos el modelo con el mismo conjunto de datos, el modelo puede aprender a predecir las etiquetas de ese conjunto de datos en lugar de generalizar a nuevos datos.

#### 2.4.3 Codificar las etiquetas

In [9]:
# Para modelos de clasificación, especialmente en Keras/TensorFlow, las etiquetas deben estar codificadas de manera que cada etiqueta sea un vector binario. Esto se conoce como codificación one-hot
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold

# Codificar las etiquetas
label_encoder = LabelEncoder()
etiquetas_encoded = label_encoder.fit_transform(etiquetas)  # Codificar todas las etiquetas
etiquetas_categorical = to_categorical(etiquetas_encoded)  # Convertir a one-hot

# Configurar validación cruzada con 5 divisiones
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Variables para almacenar los resultados de la validación cruzada
fold_no = 1
acc_per_fold = []
loss_per_fold = []

## 3. Creación del modelo

Una vez que se han preprocesado los datos, se va a crear el modelo de clasificación de imágenes de rostros humanos en función del estrés emocional que presentan. Para ello, se va a utilizar una red neuronal densa que consta de varias capas totalmente conectadas.

### 3.1 Definición de los modelos

En este apartado se van a explicar los modelos que se van a utilizar para la clasificación de las imágenes. Todos los modelos que se muestran en este apartado son redes neuronales densas.
Todos estos modelos se han elegido debido a que a lo largo del tiempo se han utilizado para diferentes objetivos de clasificación y han obtenido buenos resultados.

#### 3.1.1 Modelo Prueba

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout

# Función para crear el modelo
def create_model_1():
    model = Sequential([
        Flatten(input_shape=(256, 7, 7)),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(4, activation='softmax')  # Asume 4 clases como en tu ejemplo
    ])
    return model

#### Compilar y entrenar el modelo

In [11]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers.schedules import ExponentialDecay


# Validación cruzada, iterar sobre cada división
for train_index, val_index in kf.split(X_train):
    print(f"Fold número {fold_no}")
    # Usar las divisiones correctas para el entrenamiento y la validación
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    # Crear el modelo
    model = create_model_1()
    
    # Compilar el modelo
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Entrenar el modelo y validar con los datos de validación
    print(f'Training fold {fold_no}...')
    history = model.fit(X_train_fold, y_train_fold, epochs=10, batch_size=32, validation_data=(X_val_fold, y_val_fold))
    
    # Evaluar el modelo con los datos de test
    scores = model.evaluate(X_test, y_test, verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    #Incrementar el número de la división
    fold_no += 1

Fold número 1


2024-07-17 13:35:24.238369: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-17 13:35:24.563928: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-17 13:35:24.564178: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-17 13:35:24.564409: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

Training fold 1...


2024-07-17 13:35:25.398879: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1541406720 exceeds 10% of free system memory.
2024-07-17 13:35:26.307511: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1541406720 exceeds 10% of free system memory.


Epoch 1/10


...

960/960 [==============================] - 4s 3ms/step - loss: 0.3100 - accuracy: 0.8850 - val_loss: 0.1191 - val_accuracy: 0.9639
Epoch 2/10
960/960 [==============================] - 2s 2ms/step - loss: 0.1066 - accuracy: 0.9654 - val_loss: 0.0805 - val_accuracy: 0.9745
Epoch 3/10
960/960 [==============================] - 2s 3ms/step - loss: 0.0706 - accuracy: 0.9781 - val_loss: 0.0650 - val_accuracy: 0.9781
Epoch 4/10
960/960 [==============================] - 2s 2ms/step - loss: 0.0553 - accuracy: 0.9818 - val_loss: 0.0488 - val_accuracy: 0.9841
Epoch 5/10
960/960 [==============================] - 2s 2ms/step - loss: 0.0415 - accuracy: 0.9869 - val_loss: 0.0401 - val_accuracy: 0.9862
Epoch 6/10
960/960 [==============================] - 2s 2ms/step - loss: 0.0351 - accuracy: 0.9882 - val_loss: 0.0420 - val_accuracy: 0.9852
Epoch 7/10
960/960 [==============================] - 2s 2ms/step - loss: 0.0300 - accuracy: 0.9897 - val_loss: 0.0342 - val_accuracy: 0.9882
Epoch 8/10
960/96

2024-07-17 13:35:53.974873: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1541406720 exceeds 10% of free system memory.
2024-07-17 13:35:54.873788: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1541406720 exceeds 10% of free system memory.


Epoch 1/10
960/960 [==============================] - 3s 3ms/step - loss: 0.3064 - accuracy: 0.8884 - val_loss: 0.1235 - val_accuracy: 0.9598
Epoch 2/10
960/960 [==============================] - 2s 3ms/step - loss: 0.1042 - accuracy: 0.9660 - val_loss: 0.0758 - val_accuracy: 0.9755
Epoch 3/10
960/960 [==============================] - 2s 3ms/step - loss: 0.0694 - accuracy: 0.9780 - val_loss: 0.0585 - val_accuracy: 0.9794
Epoch 4/10
960/960 [==============================] - 2s 2ms/step - loss: 0.0539 - accuracy: 0.9827 - val_loss: 0.0511 - val_accuracy: 0.9826
Epoch 5/10
960/960 [==============================] - 2s 3ms/step - loss: 0.0421 - accuracy: 0.9865 - val_loss: 0.0393 - val_accuracy: 0.9863
Epoch 6/10
960/960 [==============================] - 2s 2ms/step - loss: 0.0347 - accuracy: 0.9884 - val_loss: 0.0371 - val_accuracy: 0.9871
Epoch 7/10
960/960 [==============================] - 2s 2ms/step - loss: 0.0269 - accuracy: 0.9917 - val_loss: 0.0337 - val_accuracy: 0.9885
Epoch 

2024-07-17 13:36:20.837327: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1541406720 exceeds 10% of free system memory.


Epoch 1/10
960/960 [==============================] - 3s 3ms/step - loss: 0.2962 - accuracy: 0.8943 - val_loss: 0.1215 - val_accuracy: 0.9630
Epoch 2/10
960/960 [==============================] - 2s 2ms/step - loss: 0.1016 - accuracy: 0.9667 - val_loss: 0.0755 - val_accuracy: 0.9772
Epoch 3/10
960/960 [==============================] - 2s 2ms/step - loss: 0.0700 - accuracy: 0.9774 - val_loss: 0.0666 - val_accuracy: 0.9770
Epoch 4/10
960/960 [==============================] - 2s 3ms/step - loss: 0.0516 - accuracy: 0.9826 - val_loss: 0.0431 - val_accuracy: 0.9862
Epoch 5/10
960/960 [==============================] - 2s 2ms/step - loss: 0.0386 - accuracy: 0.9874 - val_loss: 0.0482 - val_accuracy: 0.9840
Epoch 6/10
960/960 [==============================] - 2s 3ms/step - loss: 0.0354 - accuracy: 0.9886 - val_loss: 0.0357 - val_accuracy: 0.9887
Epoch 7/10
960/960 [==============================] - 2s 3ms/step - loss: 0.0301 - accuracy: 0.9903 - val_loss: 0.0328 - val_accuracy: 0.9902
Epoch 

#### Validación del modelo


In [12]:
# Mostrar los resultados de la validación cruzada
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

# Reiniciar las variables para almacenar los resultados de la validación cruzada de otro modelo
fold_no = 1
acc_per_fold = []
loss_per_fold = []

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.030726486817002296 - Accuracy: 99.06250238418579%
------------------------------------------------------------------------
> Fold 2 - Loss: 0.033210668712854385 - Accuracy: 98.95312786102295%
------------------------------------------------------------------------
> Fold 3 - Loss: 0.03544995188713074 - Accuracy: 98.79687428474426%
------------------------------------------------------------------------
> Fold 4 - Loss: 0.02954978495836258 - Accuracy: 99.09374713897705%
------------------------------------------------------------------------
> Fold 5 - Loss: 0.03163372352719307 - Accuracy: 98.96093606948853%
------------------------------------------------------------------------
Average scores for all folds:
> Accuracy: 98.97343754768372 (+- 0.10406757324686733)
> Loss: 0.032114123180508616
------------------

#### 3.1.2 Modelo LeNet

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout

# Asumiendo que tus datos son características extraídas (256 características por cada una de las 7x7 "imágenes")
def create_model_2():
    model_LeNet = Sequential([
        Flatten(input_shape=(256, 7, 7)), # Aplanamos las características para hacerlas compatibles con capas densas
        Dense(512, activation='relu'), # Aumentamos la dimensión. Ajusta según necesidad.
        Dropout(0.5), # Ayuda a prevenir el sobreajuste
        Dense(120, activation='relu'), # Capa densa con 120 nodos como en LeNet
        Dense(84, activation='relu'), # Capa densa con 84 nodos como en LeNet
        Dense(4, activation='softmax') # Capa de salida para 4 clases
    ])
    return model_LeNet


#### Compilar y entrenar el modelo

In [ ]:
# Antes de entrenar el modelo, debes compilarlo, especificando la función de pérdida y el optimizador que utilizarás.
# Aplicar la codificación one-hot
for train_index, test_index in kf.split(imagenes):
    print(f"Fold número {fold_no}")
    X_train, X_test = imagenes[train_index], imagenes[test_index]
    y_train, y_test = etiquetas_categorical[train_index], etiquetas_categorical[test_index]

    # Crear el modelo
    model = create_model_2()
    
    # Compilar el modelo
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Entrenar el modelo
    print(f'Training fold {fold_no}...')
    history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.1)
    
    # Evaluar el modelo con los datos de prueba
    scores = model.evaluate(X_test, y_test, verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    #Incrementar el número de la división
    fold_no += 1

Fold número 1


2024-07-17 13:30:05.981514: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-17 13:30:06.294483: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-17 13:30:06.294776: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-17 13:30:06.294995: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

Training fold 1...


2024-07-17 13:30:08.578458: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2312110080 exceeds 10% of free system memory.


Epoch 1/10


...

1440/1440 [==============================] - 6s 3ms/step - loss: 0.2273 - accuracy: 0.9050 - val_loss: 0.3440 - val_accuracy: 0.8549
Epoch 2/10
1440/1440 [==============================] - 4s 3ms/step - loss: 0.0949 - accuracy: 0.9654 - val_loss: 0.4328 - val_accuracy: 0.8383
Epoch 3/10
1440/1440 [==============================] - 3s 2ms/step - loss: 0.0860 - accuracy: 0.9685 - val_loss: 0.8030 - val_accuracy: 0.8400
Epoch 4/10
1440/1440 [==============================] - 3s 2ms/step - loss: 0.0760 - accuracy: 0.9725 - val_loss: 0.3864 - val_accuracy: 0.8514
Epoch 5/10
1440/1440 [==============================] - 4s 3ms/step - loss: 0.0662 - accuracy: 0.9762 - val_loss: 0.4257 - val_accuracy: 0.8570
Epoch 6/10
1440/1440 [==============================] - 4s 3ms/step - loss: 0.0632 - accuracy: 0.9771 - val_loss: 0.5912 - val_accuracy: 0.8318
Epoch 7/10
1440/1440 [==============================] - 4s 2ms/step - loss: 0.0599 - accuracy: 0.9789 - val_loss: 0.6588 - val_accuracy: 0.8090
Epo

2024-07-17 13:30:50.573307: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2312110080 exceeds 10% of free system memory.
2024-07-17 13:30:51.906764: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2312110080 exceeds 10% of free system memory.


Epoch 1/10
1440/1440 [==============================] - 5s 3ms/step - loss: 0.2218 - accuracy: 0.9099 - val_loss: 0.4359 - val_accuracy: 0.8541
Epoch 2/10
1440/1440 [==============================] - 4s 3ms/step - loss: 0.0958 - accuracy: 0.9658 - val_loss: 0.3700 - val_accuracy: 0.8605
Epoch 3/10
1440/1440 [==============================] - 4s 3ms/step - loss: 0.0795 - accuracy: 0.9717 - val_loss: 0.6383 - val_accuracy: 0.8463
Epoch 4/10
1440/1440 [==============================] - 4s 3ms/step - loss: 0.0742 - accuracy: 0.9733 - val_loss: 0.6876 - val_accuracy: 0.8422
Epoch 5/10
1440/1440 [==============================] - 4s 3ms/step - loss: 0.0699 - accuracy: 0.9749 - val_loss: 0.3395 - val_accuracy: 0.8580
Epoch 6/10
1440/1440 [==============================] - 4s 3ms/step - loss: 0.0581 - accuracy: 0.9791 - val_loss: 0.6770 - val_accuracy: 0.8354
Epoch 7/10
1440/1440 [==============================] - 4s 2ms/step - loss: 0.0589 - accuracy: 0.9787 - val_loss: 0.4475 - val_accuracy:

2024-07-17 13:31:34.116992: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2312110080 exceeds 10% of free system memory.


Epoch 1/10
1440/1440 [==============================] - 5s 3ms/step - loss: 0.2239 - accuracy: 0.9076 - val_loss: 0.4892 - val_accuracy: 0.8398
Epoch 2/10
1440/1440 [==============================] - 4s 3ms/step - loss: 0.0998 - accuracy: 0.9643 - val_loss: 0.4544 - val_accuracy: 0.8328
Epoch 3/10
1440/1440 [==============================] - 4s 3ms/step - loss: 0.0797 - accuracy: 0.9715 - val_loss: 0.4252 - val_accuracy: 0.8480
Epoch 4/10
1440/1440 [==============================] - 4s 3ms/step - loss: 0.0692 - accuracy: 0.9757 - val_loss: 0.5209 - val_accuracy: 0.8389
Epoch 5/10
1440/1440 [==============================] - 4s 3ms/step - loss: 0.0589 - accuracy: 0.9786 - val_loss: 0.5939 - val_accuracy: 0.8355
Epoch 6/10
1440/1440 [==============================] - 4s 3ms/step - loss: 0.0640 - accuracy: 0.9766 - val_loss: 0.8230 - val_accuracy: 0.8232
Epoch 7/10
1440/1440 [==============================] - 4s 3ms/step - loss: 0.0568 - accuracy: 0.9787 - val_loss: 0.4804 - val_accuracy:

#### Validación del modelo

In [ ]:
# Mostrar los resultados de la validación cruzada
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.10646235197782516 - Accuracy: 97.24218845367432%
------------------------------------------------------------------------
> Fold 2 - Loss: 0.08546216040849686 - Accuracy: 97.67968654632568%
------------------------------------------------------------------------
> Fold 3 - Loss: 0.09387540817260742 - Accuracy: 97.48437404632568%
------------------------------------------------------------------------
> Fold 4 - Loss: 0.07565674185752869 - Accuracy: 97.60937690734863%
------------------------------------------------------------------------
> Fold 5 - Loss: 0.0886237695813179 - Accuracy: 97.19531536102295%
------------------------------------------------------------------------
Average scores for all folds:
> Accuracy: 97.44218826293945 (+- 0.1934338497561697)
> Loss: 0.0900160863995552
------------------------